In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
df = pd.read_csv('energydata_complete.csv')
df.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [ ]:
df.columns
df.drop(columns='date', inplace=True)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
normalised_df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

C:\Users\Adbeel\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [11]:
X = normalised_df[['T2']]

Y = normalised_df['T6']

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state =1) 

In [13]:
from sklearn.linear_model import LinearRegression
linear_model = LinearRegression()
linear_model.fit(X_train, Y_train)
Y_pred = linear_model.predict(X_test)

In [14]:
from sklearn.metrics import r2_score
r2_score = r2_score(Y_test, Y_pred)
r2_score

0.6452256534282057

In [15]:
from sklearn.metrics import mean_squared_error 
rmse = np.sqrt(mean_squared_error(Y_test, Y_pred))
rmse

0.10581315813021211

In [16]:
from sklearn.linear_model import Ridge
ridge = Ridge(alpha=0.4)
ridge.fit(X_train, Y_train)

Ridge(alpha=0.4, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [17]:
from sklearn.linear_model import Ridge
ridge = Ridge(alpha=0.4)
ridge.fit(X_train, Y_train)
Y_ridge_pred = ridge.predict(X_test)


In [18]:
rmse = np.sqrt(mean_squared_error(Y_test, Y_ridge_pred))
rmse

0.10581418799428674

In [19]:
from sklearn.linear_model import Lasso
lasso_reg = Lasso(alpha=0.001)
lasso_reg.fit(X_train, Y_train)

Y_lass_pred = lasso_reg.predict(X_test)


'''Y_lasso_pred = lasso_reg.predict(X_test)
my_r2_score_lasso= 1 - (np.sum(np.square(Y_test - Y_lasso_pred)) / np.sum(np.square(Y_test - np.mean(Y_test))))
my_r2_score_lasso'''

'Y_lasso_pred = lasso_reg.predict(X_test)\nmy_r2_score_lasso= 1 - (np.sum(np.square(Y_test - Y_lasso_pred)) / np.sum(np.square(Y_test - np.mean(Y_test))))\nmy_r2_score_lasso'

In [20]:
rss = np.sum(np.square(Y_test - Y_lass_pred))
rss

55.47826482786419

In [23]:
my_r2_score = 1 - (np.sum(np.square(Y_test - Y_lass_pred)) / np.sum(np.square(Y_test - np.mean(Y_test))))
my_r2_score

0.6437157865218925

In [25]:
from sklearn.metrics import mean_absolute_error 
MAE = mean_absolute_error(Y_test, Y_lass_pred)
print(MAE)

0.08298412530598816


In [26]:
my_rmse = np.sqrt((1/len(Y_lass_pred)) * np.sum(np.square(Y_test - Y_lass_pred)))
my_rmse

0.10603808161365066

In [27]:
def get_weights_df(model, feat, col_name):
    weights = pd.Series(model.coef_, feat.columns).sort_values()
    weights_df = pd.DataFrame(weights).reset_index()
    weights_df.columns = ['Features', col_name]
    weights_df[col_name].round(3)
    return weights_df

In [29]:
linear_model_weights = get_weights_df(linear_model, X_train, 'Linear_Model_Weight')
ridge_weights_df = get_weights_df(ridge, X_train, 'Ridge_Weight')
lasso_weights_df = get_weights_df(lasso_reg, X_train, 'Lasso_Weight')

In [30]:
final_weights = pd.merge(linear_model_weights, ridge_weights_df, on='Features')
final_weights = pd.merge(final_weights, lasso_weights_df, on='Features') 
final_weights

,Features,Linear_Model_Weight,Ridge_Weight,Lasso_Weight
0,T2,0.88996,0.889013,0.850571


In [31]:
X1 = normalised_df.drop(columns='T6')

Y1 = normalised_df['T6']

In [32]:
X1.columns

Index(['Appliances', 'lights', 'T1', 'RH_1', 'T2', 'RH_2', 'T3', 'RH_3', 'T4',
       'RH_4', 'T5', 'RH_5', 'RH_6', 'T7', 'RH_7', 'T8', 'RH_8', 'T9', 'RH_9',
       'T_out', 'Press_mm_hg', 'RH_out', 'Windspeed', 'Visibility',
       'Tdewpoint', 'rv1', 'rv2'],
      dtype='object')

In [33]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X1, Y1, random_state =1) 

In [34]:
from sklearn.linear_model import Lasso
lasso = Lasso(alpha=0.001)
lasso.fit(X_train, Y_train)

Y_lass = lasso.predict(X_test)


In [37]:
lasso_df = get_weights_df(lasso, X_train, 'Lasso_Weight')
#lasso_df

In [ ]:
'''final_weights = pd.merge(linear_model_weights, ridge_weights_df, on='Features')
final_weights = pd.merge(final_weights, lasso_df, on='Features') 
final_weights'''

In [38]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(X_train, Y_train)
Y_pred = lm.predict(X_test)

In [47]:
lm_df = get_weights_df(lm, X_train, 'Linear_Weight')
lm_df

,Features,Linear_Weight
0,T1,-0.207054
1,Tdewpoint,-0.095320
2,RH_6,-0.094092
3,RH_3,-0.073827
4,T5,-0.072397
5,RH_1,-0.050541
6,RH_8,-0.045352
7,T8,-0.041364
8,T7,-0.038785
9,T4,-0.029086


In [48]:
rss = np.sum(np.square(Y_test - Y_lass))
rss

6.804918974303148

In [49]:
my_r2_score = 1 - (np.sum(np.square(Y_test - Y_lass)) / np.sum(np.square(Y_test - np.mean(Y_test))))
my_r2_score

0.9562984672995011

In [50]:
from sklearn.metrics import mean_absolute_error 
MAE = mean_absolute_error(Y_test, Y_lass)
print(MAE)

0.02798123931775184


In [51]:
from sklearn.metrics import mean_squared_error 
rmse = np.sqrt(mean_squared_error(Y_test, Y_lass))
rmse

0.037137435168034715

In [52]:
from sklearn.linear_model import Ridge
ridge = Ridge(alpha=0.4)
ridge.fit(X_train, Y_train)
Y_ridge_pred = ridge.predict(X_test)


In [53]:
from sklearn.metrics import mean_squared_error 
rmse = np.sqrt(mean_squared_error(Y_test, Y_ridge_pred))
rmse

0.030179022803319674